<a href="https://colab.research.google.com/github/GabrielFCosta/TCC-BSI-2023.1/blob/main/old/matc97_gabriel_teste_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import string
import nltk
import math
import random
import pandas as pd
import time as tm
from urllib.request import urlopen
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
# Remove pontuação, utilizada pela função seguinte de pré-processamento
def remover_pontuacao(tokens):
  for token in tokens:
    if token in string.punctuation:
      tokens.remove(token)
# Função de pré-processamento
def preprocess(text):
    text = word_tokenize(text)
    remover_pontuacao(text)
    return pos_tag(text)

# Listas para coletar rótulos e entidades (palavras) do texto
codes = []
words = []
def  clearlists():
  if len(words) > 0:
    words.clear()
  if len(codes) > 0:
    codes.clear()

# Função de reconhecimento de entidades nomeadas (NER)
def ner(sent):
  ne_tree = nltk.ne_chunk(sent)
  for chunk in ne_tree:
    if hasattr(chunk,"label"):
      word =' '.join(c[0] for c in chunk)
      #print(chunk.label(),word)
      codes.append(chunk.label())
      words.append(word)

# Função que retorna dataframe de entidades (palavras), rótulos e frequências 
def returndataset(col1,col2):
  # junta listas de palavras e rótulos num dataframe
  dataset = pd.DataFrame(zip(words,codes),columns =[col1,col2])
  clearlists()
  # calcula frequências das palavras e adiciona nova coluna
  dataset['freq'] = dataset.groupby([col1,col2])[col1].transform('count')
  return dataclean(dataset)
 
# Remove linhas duplicatas e atualiza index
def dataclean(dataset):
  dataset.drop_duplicates(inplace= True)
  dataset.reset_index()
  return dataset

# Verifica rótulo gramatical
def test(tag):
  # verificando todos substantivos, adjetivos, verbos e numerais cardinais.
  #tags = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ','CD']
  tags = ['NN','NNS','NNP','NNPS']
  for i in tags:
    if i == tag:
      return True
  return False

# Filtra palavras da lista gerada pelo pos tagging e retorna dataframe
def filtertuples(textlist,rtdata):
  for tuple in textlist:
    if test(tuple[1]):
      words.append(tuple[0])
      codes.append(tuple[1])
  if rtdata:
    return returndataset('word','pos_tag')
  else:
    aux = words.copy()
    clearlists()
  return aux

# Retorna união de dois dataframes com "frequência de documentos" 
def datamerge(words0,words1,col1,col2):
  merged = pd.merge(words0,words1, on=[col1,col2], how='outer')
  merged = dataclean(merged).fillna(0)
  merged['doc_freq'] = getdoc_freq(merged)
  return merged.sort_values(by=[col1,col2])

# Retorna lista de "frequência de documentos"
def getdoc_freq(merged):
  doc_freq = []
  c = 0
  # verifica duas colunas de frequência
  for i in merged['freq_x'].tolist():
    # se frequência maior que 0 em ambas colunas 
    if i > 0 and merged['freq_y'].tolist()[c] > 0:
      doc_freq.append(2)
    else:
      doc_freq.append(1)
    c += 1
  return doc_freq

# interseção dividida pela união sem considerar frequências
def jaccard_simples(doc_freq):
    intersecao = 0
    c = 0
    for x in doc_freq:
      if x == 2:
        intersecao += 1
      c += 1
    #print("interseção:",intersecao)
    uniao = len(doc_freq)
    #print("união:",uniao)
    print("jaccard_simples:",intersecao/uniao)

# Calculado a partir das frequências
# De acordo com https://towardsdatascience.com/nlp-text-similarity-how-it-works-and-the-math-behind-it-a0fb90a05095
def cosine_simples(mergedf):
  numerador = 0
  vetx = 0
  vety = 0
  for idx, row in mergedf.iterrows():
    numerador += row['freq_x'] * row['freq_y']
    vetx += pow(row['freq_x'],2)
    vety += pow(row['freq_y'],2)
  vetx = math.sqrt(vetx)
  vety = math.sqrt(vety)
  denominador = vetx * vety
  print("cosine_simples:",numerador/denominador)
  

In [ ]:
# Similaridade com base na média das médias. Não utilizar, caso base != 1
def similaridade_WP_means(str0,str1):
  means = []
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se synset0 contém alguma definição
    if len(syn0) > 0:
      val = 0
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se synset1 contém alguma definição
        if len(syn1) > 0:
          val+= syn0[0].wup_similarity(syn1[0])
          #print("syn0:",syn0[0],"syn1:",syn1[0],"val:",val)
      means.append(val/len(str1))
  #print(means)
  print('média das médias WP:',sum(means)/len(means))

# Similaridade com base na média das similaridades máximas entre palavras
def similaridade_WP_max(str0,str1):
  means = []
  vals = []
  aux = 0
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se 1a palavra contém alguma definição antes de prosseguir
    if len(syn0) != 0:
      vals.clear()
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se 2a palavra contém alguma definição antes de calcular similaridade
        if len(syn1) != 0:
          aux = syn0[0].wup_similarity(syn1[0])
          # se similaridade = 1, palavras iguais, break
          if aux == 1:
            break
          # senão, adiciona valor à lista vals
          else:
            vals.append(aux)
            #print("syn0:",syn0[0],"syn1:",syn1[0],"wup:",syn0[0].wup_similarity(syn1[0]))
      # se aux = 1, similaridade máxima = 1
      if aux == 1:
         means.append(aux)
      # senão seleciona máxima da lista
      else:
        means.append(max(vals))
  #print(means)
  print('média das máximas WP:',sum(means)/len(means))


In [ ]:
# Retorna lemmas e hiperônimos das palavras contidas na lista do parâmetro
def getclasses(stringlist):
  lemmas = []
  hypernyms = []
  na = 'N\A'
  for wd in stringlist:
    syn = wn.synsets(wd)
    if len(syn) > 0:
      lemmas.append(syn[0].lemmas()[0].name())
      if len(syn[0].hypernyms()) > 0:
        hypernyms.append(syn[0].hypernyms()[0].name())
      else:
        hypernyms.append(na)
    else:
      lemmas.append(na)
      hypernyms.append(na)
  return pd.DataFrame(zip(stringlist,lemmas,hypernyms),columns =['word','lemma','hypernym'])

# Retorna tabela de frequências do tipo de classe (lemma ou hypernym), passado no parâmetro col
def intersects(df0,df1,col):
  aux = []
  for i in df0.tolist():
    if i != 'N\A':
      aux.append(i)
  df0 = pd.DataFrame(zip(aux),columns =[col])
  df0['freq'] = df0.groupby([col])[col].transform('count')
  #print(df0)
  aux.clear()
  for i in df1.tolist():
    if i != 'N\A':
      aux.append(i)
  df1 = pd.DataFrame(zip(aux),columns =[col])
  df1['freq'] = df1.groupby([col])[col].transform('count')
  #print(df1)
  return mergeclass(df0,df1,col)

# Auxiliar à função intersects pra fazer o merge das tabelas do parâmetro
def mergeclass(df0,df1,col):
  merged = pd.merge(df0,df1, on=[col], how='outer')
  merged = dataclean(merged).fillna(0)
  merged['doc_freq'] = getdoc_freq(merged)
  return merged.sort_values(by=[col])


In [ ]:
# Execução do reconhecimento de entidades nomeadadas:
'''
urls = ["https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia.txt",
        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia1.txt",
        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/noticia2.txt"]
# Lista que vai conter dataframes de entidades
datasets = []

# Pra cada url
for url in urls:
  data = urlopen(url)
  ex = data.read().decode('utf-8')
  ex = nltk.sent_tokenize(url)
  print(ex)
  # Detectando entidades de nomeadas por sentença
  for sentence in ex:
    # Sentença Pré-processada
    text = preprocess(sentence)
    print(text)
    # Entidades Nomeadas na Sentença
    ner(text)
  # Adiciona dataframe de entidades nomeadas do texto + labels, frequências e TF
  datasets.append(returndataset('entity','label'))

c = 0
for dataset in datasets:
  print("\nstring:",c)
  print(dataset)
  c += 1
'''
frases = ["Airbnb will shut down its listings in China after two years of lockdowns in the country.",
        "Starting this summer, Airbnb will take down its listings and offers for hosted experiences in China.",
        "International brands, from Apple to Estee Lauder, have warned of the financial impact of the restrictions."]

def compnomeadas(frase0,frase1):
  ner(preprocess(frase0))
  dt0 = returndataset('entity','label')
  ner(preprocess(frase1))
  dt = datamerge(dt0,returndataset('entity','label'),'entity','label')
  #print(dt)
  jaccard_simples(dt['doc_freq'].tolist())
  cosine_simples(dt)

def comppostags(frase0,frase1):
  str0 = filtertuples(preprocess(frase0),True)
  str1 = filtertuples(preprocess(frase1),True)
  #print("string 0:",str0['word'].tolist())
  #print("string 1:",str1['word'].tolist())
  similaridade_WP_max(str0['word'].tolist(),str1['word'].tolist())
  dt = datamerge(str0,str1,'word','pos_tag')
  jaccard_simples(dt['doc_freq'].tolist())
  cosine_simples(dt)

def compwordnet(frase0,frase1,classe):
  #print(preprocess(frase0))
  print(filtertuples(preprocess(frase0),False))
  str0 = getclasses(filtertuples(preprocess(frase0),False))
  print(str0,'\n')
  str1 = getclasses(filtertuples(preprocess(frase1),False))
  print(str1,'\n')
  df = intersects(str0[classe],str1[classe],classe)
  print(df)
  jaccard_simples(df['doc_freq'].tolist())
  cosine_simples(df)

In [ ]:
start = tm.time()
print("\n######## Comparação de Entidades Nomeadas ########")
print("\n## strings 0 e 0 (caso base): ##")
compnomeadas(frases[0],frases[0])
print("\n## strings 0 e 1: ##")
compnomeadas(frases[0],frases[1])
print("\n## strings 0 e 2: ##")
compnomeadas(frases[0],frases[2])
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

start = tm.time()
print("\n######## Comparação de POS TAGS ########")
print("\n## strings 0 e 0 (caso base): ##")
comppostags(frases[0],frases[0])
print("\n## strings 0 e 1: ##")
comppostags(frases[0],frases[1])
print("\n## strings 0 e 2: ##")
comppostags(frases[0],frases[2])
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

# cosine do scikit
start = tm.time()
print('\n######## Cosine do Scikit ########')
print("\n## strings 0 e 1: ##")
corpus = [frases[0],frases[1]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
print("\n## strings 0 e 2: ##")
corpus = [frases[0],frases[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
print("\n## strings 0, 1 e 2: ##")
corpus = [frases[0],frases[1],frases[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

start = tm.time()
print('\n######## Comparação de Lemmas do Wordnet ########')
print("\n## strings 0 e 0 (caso base): ##")
compwordnet(frases[0],frases[0],'lemma')
print("\n## strings 0 e 1: ##")
compwordnet(frases[0],frases[1],'lemma')
print("\n## strings 0 e 2: ##")
compwordnet(frases[0],frases[2],'lemma')
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

start = tm.time()
print('\n######## Comparação de Hiperônimos do Wordnet ########')
print("\n## strings 0 e 0 (caso base): ##")
compwordnet(frases[0],frases[0],'hypernym')
print("\n## strings 0 e 1: ##")
compwordnet(frases[0],frases[1],'hypernym')
print("\n## strings 0 e 2: ##")
compwordnet(frases[0],frases[2],'hypernym')
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")



######## Comparação de Entidades Nomeadas ########

## strings 0 e 0 (caso base): ##
jaccard_simples: 1.0
cosine_simples: 0.9999999999999998

## strings 0 e 1: ##
jaccard_simples: 0.3333333333333333
cosine_simples: 0.4999999999999999

## strings 0 e 2: ##
jaccard_simples: 0.0
cosine_simples: 0.0

######## Tempo de execução: 0.38097310066223145 ########

######## Comparação de POS TAGS ########

## strings 0 e 0 (caso base): ##
média das máximas WP: 1.0
jaccard_simples: 1.0
cosine_simples: 1.0000000000000002

## strings 0 e 1: ##
média das máximas WP: 0.7073015873015873
jaccard_simples: 0.3333333333333333
cosine_simples: 0.5000000000000001

## strings 0 e 2: ##
média das máximas WP: 0.3337628384687208
jaccard_simples: 0.0
cosine_simples: 0.0

######## Tempo de execução: 2.3257617950439453 ########

######## Cosine do Scikit ########

## strings 0 e 1: ##
[[1.         0.31639145]]

## strings 0 e 2: ##
[[1.         0.12254598]]

## strings 0, 1 e 2: ##
[[1.         0.35608377 0.15772405

In [ ]:
# retorna todos elementos da string numa lista
def retstringlist(string):
  textlist = preprocess(string)
  aux = []
  for tuple in textlist:
    aux.append(tuple[0])
  return aux
# converte lista pra dataframe de frequências
def returndatasimples(aux):
  # junta listas de palavras e rótulos num dataframe
  dataset = pd.DataFrame(aux,columns =['elementos'])
  # calcula frequências das palavras e adiciona nova coluna
  dataset['freq'] = dataset.groupby(['elementos'])['elementos'].transform('count')
  return dataclean(dataset)

df0 = returndatasimples(retstringlist(frases[0]))
df1 = returndatasimples(retstringlist(frases[1]))
df2 = returndatasimples(retstringlist(frases[2]))
cosine_simples(mergeclass(df0,df0,'elementos'))
cosine_simples(mergeclass(df0,df1,'elementos'))
cosine_simples(mergeclass(df0,df2,'elementos'))

cosine_simples: 1.0000000000000002
cosine_simples: 0.47140452079103173
cosine_simples: 0.21081851067789195


In [ ]:
urls = ["https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/movies.csv",
        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/ratings.csv",
        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/tags.csv"]
datasets = []
auxlist =[]
for url in urls:
  datasets.append(pd.read_csv(url))
# concatena títulos e gêneros na coluna string
datasets[0]['string'] = datasets[0]['title'] + " " + datasets[0]['genres']
# substitui barras verticais '|' por espaços entre gêneros 
aux =''
for idx1, i in datasets[0].iterrows():
  aux = i['string'].replace('|', " ")
  auxlist.append(aux)
datasets[0]['string'] = auxlist
auxlist.clear()
# concatena tags de usuários
for idx1, i in datasets[0].iterrows():
  df = datasets[2].loc[datasets[2]['movieId'] == i['movieId'] ] 
  aux =''
  for idx2, j in df.iterrows():
    aux = aux +" "+ j['tag']
  #print("movie:"+str(i['movieId'])+" tags:"+aux)
  auxlist.append(aux)
datasets[0]['string'] = datasets[0]['string'] + "" + auxlist



In [ ]:
# filtra todos os filmes com rating máximo de determninado usuário
id = random.randrange(1, 610)
user = datasets[1].loc[(datasets[1]['userId'] == id) & (datasets[1]['rating'] == 5.0)]
if user.shape[0] > 10:
  # sorteia 11 filmes do usuário
  usersample = user.sample(n=11)

  # pega todos os ids de filmes
  mids = datasets[0]['movieId']
  # exclui os 11 filmes do usuário do total de filmes
  mids = mids[mids.isin(usersample['movieId'].tolist()) == False]
  # sorteia 40 outros filmes 
  mids = mids.sample(n=40)
  mids = mids.tolist()

  # desses 11, sorteia 1 para servir de referência e o exclui dos outros 11
  reference = usersample.sample()
  refmovie = reference['movieId'].iloc[0]
  index = usersample[usersample['movieId'] == refmovie ].index
  usersample = usersample.drop(index)
  usersample = usersample['movieId'].tolist()

  # junta duas listas de 10 filmes de rating 5 com outros 40 aleatórios
  mids = mids + usersample
  print(refmovie)
  print(len(mids))
  # pega filme de referência na base de filmes
  aux = datasets[0][datasets[0]['movieId'] == refmovie]
  #print(preprocess(aux['string'].iloc[0]))
# usuário deve ter mais de 10 filmes com rating 5
else:
  print("Usuário não avaliou filmes suficientes.")

6333
50
